<font size=5 face='黑体'>关于pandas的分类方法

In [2]:
import pandas as pd

s = pd.Series(['a','b','c','d'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [4]:
#此时cat_s还是Series，但cat_s.values是category
cat_s.values.codes

array([0, 1, 2, 3, 0, 1, 2, 3], dtype=int8)

In [5]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

<font size=5 face='黑体'>statsmodels——估计线性模型

In [2]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as  smf

In [4]:
def dnorm(mean,variance,size=1):
    if isinstance(size,int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

np.random.seed(12345)

N = 100
X = np.c_[dnorm(0,0.4,size=N),dnorm(0,0.6,size=N),dnorm(0,0.2,size=N)]
eps = dnorm(0,0.1,size=N)
beta = [0.1,0.3,0.5]

y = np.dot(X,beta) + eps

In [6]:
X.shape

(100, 3)

In [8]:
beta.shape

AttributeError: 'list' object has no attribute 'shape'

In [9]:
np.dot(X,beta).shape

(100,)

In [10]:
X_model = sm.add_constant(X)   #添加截距项
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [11]:
#普通最小二乘回归
model = sm.OLS(y,X)
results = model.fit()
results.params

array([0.17826108, 0.22303962, 0.50095093])

In [12]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Thu, 31 Jan 2019   Prob (F-statistic):           7.44e-12
Time:                        16:21:13   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.0

In [13]:
#当所有的模型参数都在一个DataFrame中
import pandas as pd

data = pd.DataFrame(X,columns=['col0','col1','col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [14]:
#使用statsmodels的公式API和Patsy的公式字符串
results = smf.ols('y ~ col0 + col1 + col2',data=data).fit()
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [15]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [16]:
#预测
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

<font size=5 face='黑体'>statsmodels——估计时间序列过程

In [17]:
#模拟一个AR（2）时间序列数据
init_x = 4

import random
values = [init_x,init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0,0.1,N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [18]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)
results.params
#结果中的估计参数首先是截距，其次是前两个参数的估计值

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])